<a href="https://colab.research.google.com/github/RedietNegash/RAG/blob/main/ResearchpaperRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install langchain
!pip install -U sentence-transformers
!pip install qdrant-client
!pip install google-generativeai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/

In [ ]:
from datasets import load_dataset

ds = load_dataset("CShorten/ML-ArXiv-Papers")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

ML-Arxiv-Papers.csv:   0%|          | 0.00/147M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117592 [00:00<?, ? examples/s]

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 117592
    })
})


In [ ]:
train_dataset = ds['train']
print(train_dataset[0])

{'Unnamed: 0.1': 0, 'Unnamed: 0': 0.0, 'title': 'Learning from compressed observations', 'abstract': '  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an information-theoretic characterization of achievable predictor\nperformance in terms of conditional distortion-rate f

In [ ]:
filtered_dataset = train_dataset.remove_columns(['Unnamed: 0.1', 'Unnamed: 0'])
print(filtered_dataset[0])

{'title': 'Learning from compressed observations', 'abstract': '  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an information-theoretic characterization of achievable predictor\nperformance in terms of conditional distortion-rate functions. The ideas are\nillustrated o

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import unicodedata
import re
import string


nltk.download('stopwords')
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def process_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = " ".join([word for word in text.split() if word not in stop_words])
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


abstracts = [entry['abstract'].replace("\n", "").strip() for entry in train_dataset]
processed_abstracts = [process_text(abstract) for abstract in abstracts]


titles = [entry['title'].replace("\n", "").strip() for entry in train_dataset]


print("Processed Abstracts:", processed_abstracts[:5])



Processed Abstracts: ['problem statistical learning construct predictor randomvariable function related random variable x basis aniid training sample joint distribution xy allowablepredictors drawn specified class goal approachasymptotically performance expected loss best predictor theclass consider setting one perfect observation thexpart sample ypart communicated somefinite bit rate encoding yvalues allowed depend thexvalues suitable regularity condition admissible predictorsthe underlying family probability distribution loss function wegive informationtheoretic characterization achievable predictorperformance term conditional distortionrate function idea areillustrated example nonparametric regression gaussian noise', 'sensor network practice communication among sensor subjectto error failure random time cost constraint sincesensors network operate scarce resource power data rateor communication signaltonoise ratio snr usually main factor indetermining probability error communicatio

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [ ]:
chunked_abstracts = [text_splitter.split_text(abstract) for abstract in processed_abstracts]

In [ ]:
from sentence_transformers import SentenceTransformer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
import pandas as pd

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
data = []


max_chunks = 400
processed_chunks = 0

In [ ]:
for title, abstract in zip(titles, abstracts):
    if processed_chunks >= max_chunks:
        break

    chunks = text_splitter.split_text(abstract)

    for chunk in chunks:
        if processed_chunks >= max_chunks:
            break



        embedding = model.encode([chunk])[0]

        data.append({
            'title': title,
            'chunk_text': chunk,
            'embedding': embedding
        })

        processed_chunks += 1


df = pd.DataFrame(data)




In [ ]:
print(df.head(50))

                                                title  \
0               Learning from compressed observations   
1               Learning from compressed observations   
2               Learning from compressed observations   
3               Learning from compressed observations   
4               Learning from compressed observations   
5               Learning from compressed observations   
6               Learning from compressed observations   
7   Sensor Networks with Random Links: Topology De...   
8   Sensor Networks with Random Links: Topology De...   
9   Sensor Networks with Random Links: Topology De...   
10  Sensor Networks with Random Links: Topology De...   
11  Sensor Networks with Random Links: Topology De...   
12  Sensor Networks with Random Links: Topology De...   
13  Sensor Networks with Random Links: Topology De...   
14  Sensor Networks with Random Links: Topology De...   
15  Sensor Networks with Random Links: Topology De...   
16  Sensor Networks with Random

In [ ]:
from qdrant_client import models, QdrantClient



In [ ]:
qdrant= QdrantClient(":memory:")

In [ ]:
qdrant.recreate_collection(
    collection_name="research_papers",
    vectors_config=models.VectorParams(
        size=model.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)


<ipython-input-18-94f038011b8d>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
records = [
    models.Record(
        id=idx,
        vector=row['embedding'],
        payload={"title": row['title'], "chunk_text": row['chunk_text']}
    )
    for idx, row in df.iterrows()
]

In [ ]:
qdrant.upload_records(
    collection_name="research_papers",
    records=records
)

<ipython-input-20-4cf98bc02fae>:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [ ]:
print(f"Uploaded {len(records)} records to Qdrant")

Uploaded 400 records to Qdrant


In [ ]:
scroll_results, next_page = qdrant.scroll(collection_name="research_papers", limit=10)


for result in scroll_results:
    print(f"ID: {result.id}, Title: {result.payload['title']}, Chunk: {result.payload['chunk_text']}")


ID: 0, Title: Learning from compressed observations, Chunk: The problem of statistical learning is to construct a predictor of a randomvariable $Y$ as a function of a related random variable $X$ on the basis of ani.i.d. training sample from the joint
ID: 1, Title: Learning from compressed observations, Chunk: basis of ani.i.d. training sample from the joint distribution of $(X,Y)$. Allowablepredictors are drawn from some specified class, and the goal is to approachasymptotically the performance (expected
ID: 2, Title: Learning from compressed observations, Chunk: approachasymptotically the performance (expected loss) of the best predictor in theclass. We consider the setting in which one has perfect observation of the$X$-part of the sample, while the $Y$-part
ID: 3, Title: Learning from compressed observations, Chunk: of the$X$-part of the sample, while the $Y$-part has to be communicated at somefinite bit rate. The encoding of the $Y$-values is allowed to depend on the$X$-values. Unde

In [ ]:
def retrieve_relevant_chunks(query, model, qdrant, top_k=5):

    query_embedding = model.encode([query])[0]


    search_results = qdrant.search(
        collection_name="research_papers",
        query_vector=query_embedding,
        limit=top_k
    )


    retrieved_chunks = [result.payload['chunk_text'] for result in search_results]
    return retrieved_chunks

In [ ]:
api_key=''

In [ ]:
import google.generativeai as genai

def generate_answer_gemini(query, relevant_chunks):
    api_key = api_keyy
    genai.configure(api_key=api_key)

    context = "\n\n".join(relevant_chunks)

    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    prompt = f"Context: {context}\n\nQuestion: {query}"

    response = model.generate_content([prompt])

    if response.candidates:
        generated_text = response.candidates[0].content.parts[0].text
        return generated_text
    else:
        return "No content generated."

In [ ]:
def rag_pipeline(query, model, qdrant):

    relevant_chunks = retrieve_relevant_chunks(query, model, qdrant)


    answer = generate_answer_gemini(query, relevant_chunks)

    return answer


In [ ]:
query = "What is the problem of statistical learning"


answer = rag_pipeline(query, model, qdrant)


print("Result:", answer)


Result: The problem of statistical learning, as described in the text, is to **construct a predictor for a random variable Y based on the information from a related random variable X**.  This is done using an independent and identically distributed (i.i.d.) training sample drawn from the joint distribution of X and Y.

In simpler terms, you want to build a model that can predict the value of Y (the output) given the value of X (the input). This model is built by learning patterns from the training data, which is a set of examples where both X and Y are known.

Here's a breakdown:

* **Random variable Y:** The variable you want to predict.
* **Random variable X:** The variable that provides information about Y.
* **Training sample:** A set of examples where both X and Y are observed.
* **Predictor:** The function that maps X to Y based on the patterns learned from the training data.

The goal of statistical learning is to create a predictor that performs well on unseen data (data not in

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def evaluate_rag_system(queries, expected_answers, model, qdrant):
    generated_answers = []
    for query in queries:
        answer = rag_pipeline(query, model, qdrant)
        generated_answers.append(answer)


    accuracy = accuracy_score(expected_answers, generated_answers)
    precision, recall, f1, _ = precision_recall_fscore_support(expected_answers, generated_answers, average='macro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

sample_queries = [
    "What is the problem of statistical learning",
    "Explain the concept of deep learning"
]

expected_answers = [
    "Statistical learning addresses the problem of making predictions or decisions based on data. It deals with the challenges of model selection, parameter estimation, and understanding the underlying data distributions to improve prediction accuracy.",
    "Deep learning is a subset of machine learning that uses neural networks with multiple layers to learn representations of data. It focuses on training models to recognize patterns and make decisions through hierarchical feature extraction and representation learning."
]


evaluate_rag_system(sample_queries, expected_answers, model, qdrant)

Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
